In [10]:
import pandas as pd
import plotly.express as px
from ipywidgets import widgets, Output, VBox
from IPython.display import display

# 输出区，用于显示图表和提示
output = Output()

# 下拉菜单，选择图表类型
dropdown = widgets.Dropdown(
    options=[
        '吸烟频率年龄分布',
        '饮酒频率年龄分布',
        '心理健康与吸烟相关性',
        '研究领域饮酒对比'
    ],
    description='图表类型:'
)

# 生成图表按钮
button = widgets.Button(description='生成图表', button_style='success')

# 把控件放到一个竖直容器中
container = VBox([dropdown, button, output])

# 缓存数据，避免重复读取
data_cache = {}

def loaddata():
    try:
        if 'main' not in data_cache:
            main_df = pd.read_excel(
                'Student Smoking, Alcohol, and Mental Health Data(1)(1).xlsx',
                sheet_name='Student Smoking, Alcohol, and M'
            )
            # 填充缺失值
            main_df['吸烟频率'] = main_df['吸烟频率'].fillna('不吸烟')
            main_df['酒精消费频率'] = main_df['酒精消费频率'].fillna('不饮酒')
            main_df['心理健康'] = main_df['心理健康'].fillna('未知')
            main_df['研究领域'] = main_df.get('研究领域', pd.Series(['未知']*len(main_df)))

            data_cache['main'] = main_df
        return True
    except Exception as e:
        with output:
            output.clear_output()
            print(f"数据加载失败: {str(e)}")
        return False

def generate_chart(chart_type):
    if not loaddata():
        return
    
    df = data_cache['main']
    
    with output:
        output.clear_output()
        
        if chart_type == '吸烟频率年龄分布':
            data = df.groupby(['年龄', '吸烟频率']).size().reset_index(name='人数')
            fig = px.bar(
                data, x='年龄', y='人数', color='吸烟频率',
                title='不同年龄段吸烟频率分布',
                labels={'年龄': '年龄区间', '人数': '人数', '吸烟频率': '吸烟频率'},
                barmode='stack',
                hover_data={'年龄': True, '人数': True, '吸烟频率': True}
            )
            fig.update_layout(legend_title_text='吸烟频率')

        elif chart_type == '饮酒频率年龄分布':
            data = df.groupby(['年龄', '酒精消费频率']).size().reset_index(name='人数')
            fig = px.bar(
                data, x='年龄', y='人数', color='酒精消费频率',
                title='不同年龄段饮酒频率分布',
                labels={'年龄': '年龄区间', '人数': '人数', '酒精消费频率': '饮酒频率'},
                barmode='stack',
                hover_data={'年龄': True, '人数': True, '酒精消费频率': True}
            )
            fig.update_layout(legend_title_text='饮酒频率')

        elif chart_type == '心理健康与吸烟相关性':
            data = df.groupby(['吸烟频率', '心理健康']).size().reset_index(name='人数')
            fig = px.bar(
                data, x='吸烟频率', y='人数', color='心理健康',
                title='心理健康与吸烟行为相关性',
                labels={'吸烟频率': '吸烟频率', '人数': '人数', '心理健康': '心理健康状态'},
                barmode='stack',
                hover_data={'吸烟频率': True, '人数': True, '心理健康': True}
            )
            fig.update_layout(legend_title_text='心理健康状态')

        elif chart_type == '研究领域饮酒对比':
            if '研究领域' in df.columns:
                data = df.groupby(['研究领域', '酒精消费频率']).size().reset_index(name='人数')
                fig = px.bar(
                    data, x='研究领域', y='人数', color='酒精消费频率',
                    title='不同研究领域饮酒行为对比',
                    labels={'研究领域': '研究领域', '人数': '人数', '酒精消费频率': '饮酒频率'},
                    barmode='stack',
                    hover_data={'研究领域': True, '人数': True, '酒精消费频率': True}
                )
                fig.update_layout(legend_title_text='饮酒频率')
            else:
                print("数据中缺少 '研究领域' 列，无法生成该图表。")
                return
        fig.show()

# 点击按钮触发生成图表
button.on_click(lambda b: generate_chart(dropdown.value))

# 显示控件
display(container)

# 预加载数据
loaddata()


True